## Introduction
Essay grading is costly and time consuming for humans to do. Often in standardized tests (e.g. SAT), multiple graders are required for each essay. Our aim is to develop an automatic scoring algorithm that delivers scores close to those of human expert graders, so it can replace all/part of human graders.

Some of the potential challenges that we may face are the following:
- How to convert language into math-friendly format.
- How to deal with high dimensionality
- How to take into account the different prompts and question types that each essay has.

Below, we provide summary of the approaches we took. Please click on each chapter's title for details.

<h2 style="color:#0000EE; text-decoration:underline; display:inline" onclick="includeContent('data_exploration.html'); document.getElementById('topnv').selectedIndex=1; document.getElementById('botnav').selectedIndex=1;"> Chapter 1 - Data Exploration</h2>

We worked on a publicly available data from the Hewlett Foundation. The data contained 12976 essays from 8 different sets.

We extracted some basic statistics from the data as well as plotted histograms and scatter plots of selected characteristics to gain insights of the data.


<img src='figures/length_hist_overview.png' title='Histogram for essay length'>

<h2 style="color:#0000EE; text-decoration:underline" onclick="includeContent('baseline.html'); document.getElementById('topnv').selectedIndex=2; document.getElementById('botnav').selectedIndex=2;"> Chapter 2 - Data Cleaning: tf-idf </h2>
In order to turn essays (text data) into computer-friendly format, we used a technique called *tf-idf*. This method takes a set of text as an input, and outputs a vector of numbers that correspond to each document. When vectorizing, it penalizes words that appear often in all documents, so that frequently used words (*the, we* etc.) will have smaller weight compared to rarely used words.

<h2 style="color:#0000EE; text-decoration:underline" onclick="includeContent('baseline.html'); document.getElementById('topnv').selectedIndex=3; document.getElementById('botnav').selectedIndex=3;"> Chapter 3 - Baseline Model </h2>
Using the resultant matrices from *tf-idf*, we formed a baseline model using multiple linear regression.


<img src='figures/baseline_overview.png'>
The baseline resulted in a poor fit. In next few chapters, we worked on improving the model by dimension reduction, adding in meta features, and applying regularizations.

<h2 style="color:#0000EE; text-decoration:underline" onclick="includeContent('lsa.html'); document.getElementById('topnv').selectedIndex=4; document.getElementById('botnav').selectedIndex=4;"> Chapter 4 - LSA </h2>

After *tf idf*, we have a huge sparse matrix of vectorized text, which may have resulted in overfitting in our baseline model. Thus, we chose to perform dimension reduction to reduce the total number of features. Specifically, we used a method called *latent semantic analysis (LSA)* for this. *LSA* is essentially a PCA for textual data.

We performed a cross-validation to choose the optimal number of dimension ($d$) for each essay set.

<img src='figures/d_tuning.png'>

<h2 style="color:#0000EE; text-decoration:underline" onclick="includeContent('meta_features.html'); document.getElementById('topnv').selectedIndex=5; document.getElementById('botnav').selectedIndex=5;"> Chapter 5 - Meta Features </h2>

To improve the model, we also added in several meta-features:
- Similarity of the essay to the prompt
- Essay length in words
- Mean word length
- Mean sentence length
- Number of unique words

We checked that some of hte meta-features had correlation to the score, so we can expect them to be useful for forming the final model.

<img src='figures/corr_length_overview.png'>

<h2 style="color:#0000EE; text-decoration:underline" onclick="includeContent('lasso.html'); document.getElementById('topnv').selectedIndex=6; document.getElementById('botnav').selectedIndex=6;"> Chapter 6 - Regularization: Lasso </h2>
Even after dimension reduction using LSA, our model still had large number of predictors. We applied lasso regression to avoid overfitting and further improving the prediction accuracy.

We performed cross-validation to decide the optimal regularization parameter.

<h2 style="color:#0000EE; text-decoration:underline" onclick="includeContent('final.html'); document.getElementById('topnv').selectedIndex=7; document.getElementById('botnav').selectedIndex=7;"> Chapter 7 - Final Model </h2>

<h2 style="color:#0000EE; text-decoration:underline" onclick="includeContent('futre.html'); document.getElementById('topnv').selectedIndex=8; document.getElementById('botnav').selectedIndex=8;"> Chapter 8 - Future Work </h2>